<a href="https://colab.research.google.com/github/powersurge8891/Data-mining-with-Python/blob/main/ANALYSING_POLICE_BIAS_WITH_CHAID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#libraries and data

In [1]:
%cd /content/drive/MyDrive/Data Mining for Business in Python

/content/drive/MyDrive/Data Mining for Business in Python


In [ ]:
!pip install chaid
!pip install orca
!pip install graphviz

In [ ]:
!pip install plotly>=4.0.0
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

In [4]:
#import libraries
import pandas as pd
import statsmodels.api as sm 
from CHAID import Tree

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
#get data
data= sm.datasets.get_rdataset("MplsStops",
                               package = "carData").data 
data.head()

,idNum,date,problem,MDC,citationIssued,personSearch,vehicleSearch,preRace,race,gender,lat,long,policePrecinct,neighborhood
6823,17-000003,2017-01-01 00:00:42,suspicious,MDC,NaN,NO,NO,Unknown,Unknown,Unknown,44.966617,-93.246458,1,Cedar Riverside
6824,17-000007,2017-01-01 00:03:07,suspicious,MDC,NaN,NO,NO,Unknown,Unknown,Male,44.980450,-93.271340,1,Downtown West
6825,17-000073,2017-01-01 00:23:15,traffic,MDC,NaN,NO,NO,Unknown,White,Female,44.948350,-93.275380,5,Whittier
6826,17-000092,2017-01-01 00:33:48,suspicious,MDC,NaN,NO,NO,Unknown,East African,Male,44.948360,-93.281350,5,Whittier
6827,17-000098,2017-01-01 00:37:58,traffic,MDC,NaN,NO,NO,Unknown,White,Female,44.979078,-93.262076,1,Downtown West


#wrangling data

In [6]:
#checking the unique values
data.apply(pd.Series.nunique)

idNum             51920
date              51873
problem               2
MDC                   2
citationIssued        2
personSearch          2
vehicleSearch         2
preRace               8
race                  8
gender                3
lat                8749
long               9158
policePrecinct        5
neighborhood         87
dtype: int64

In [7]:
#removing  unnecessary columns
data=data.loc[:, ['problem', 'vehicleSearch', 'race','gender', 'policePrecinct']]

In [8]:
data.apply(pd.Series.nunique)

problem           2
vehicleSearch     2
race              8
gender            3
policePrecinct    5
dtype: int64

In [9]:
data.dtypes

problem           object
vehicleSearch     object
race              object
gender            object
policePrecinct     int64
dtype: object

#Transforming dataset

In [10]:
import seaborn as sns

In [11]:
data=pd.get_dummies(data, drop_first=True)
data.head()

,policePrecinct,problem_traffic,vehicleSearch_YES,race_Black,race_East African,race_Latino,race_Native American,race_Other,race_Unknown,race_White,gender_Male,gender_Unknown
6823,1,0,0,0,0,0,0,0,1,0,0,1
6824,1,0,0,0,0,0,0,0,1,0,1,0
6825,5,1,0,0,0,0,0,0,0,1,0,0
6826,5,0,0,0,1,0,0,0,0,0,1,0
6827,1,1,0,0,0,0,0,0,0,1,0,0


#CHAID

In [12]:
y_value=data.vehicleSearch_YES.name
x_value= data.drop(columns='vehicleSearch_YES').columns
print(x_value)
len(x_value)

Index(['policePrecinct', 'problem_traffic', 'race_Black', 'race_East African',
       'race_Latino', 'race_Native American', 'race_Other', 'race_Unknown',
       'race_White', 'gender_Male', 'gender_Unknown'],
      dtype='object')


11

In [16]:
model= Tree.from_pandas_df(data, dict(zip(x_value, ['nominal']*11)), y_value, max_depth=2)

In [15]:
import orca
import plotly
import plotly.graph_objects as go

In [17]:
#model.render()